In [1]:
import numpy as np    
import uproot3
import matplotlib.pyplot as plt

In [7]:
%matplotlib widget

In [3]:
infile_bitumen = 'data/largedrum_onlybitumen_dense_newmetrics_3cmVoxel_April2021.discriminator.root'

In [12]:
infile_21l = 'data/largedrum_21L_dense_MedianCut_3cmVoxel_all.discriminator.root'

In [4]:
fu = uproot3.open(infile_bitumen)
histogram_bitumen = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_bitumen.shape

In [13]:
fu = uproot3.open(infile_21l)
histogram_bubble = fu['histMedianMetric'].numpy()[0]
bins_x, bins_y, bins_z = fu['histMedianMetric;1'].numpy()[1][0]
shape_x, shape_y, shape_z = histogram_bubble.shape

In [5]:
def cube(x, y, z):
    if x in [0,shape_x-1] or y in [0, shape_y-1] or z in [0, shape_z-1]:
        return None
    mgrid = np.mgrid[x-1:x+2:1, y-1:y+2:1, z-1:z+2:1]
    x, y, z = map(np.ravel, mgrid)
    return x, y, z

In [6]:
def stats_count(histogram, threshold=11.265, centre='above'):
    stats_above = dict()
    stats_below = dict()
    if centre == 'above':
        xx, yy, zz = np.where(histogram>=threshold)
    elif centre == 'below':
        xx, yy, zz = np.where(histogram<threshold)
    for x, y, z in zip(xx, yy, zz):
        cubes_coord = cube(x, y, z)
        if cubes_coord is None:
            continue
        xs, ys, zs = cubes_coord
        triplets = np.array([(i, j, k) for (i, j, k) in zip(xs, ys, zs)])
        cube_around_pt = np.array([histogram[cx, cy, cz] for (cx, cy, cz) in triplets])

        cube_above = cube_around_pt[cube_around_pt >= threshold]
        cube_below = cube_around_pt[cube_around_pt < threshold]
        if centre == 'above':
            above_threshold_count = (len(cube_above) -1) if (len(cube_above) > 0) else 0.
            below_threshold_count = len(cube_below)
        elif centre== 'below':
            above_threshold_count = len(cube_above)
            below_threshold_count = (len(cube_below) -1) if (len(cube_below) > 0) else 0.
        stats_above[(x, y, z)] = above_threshold_count
        stats_below[(x, y, z)] = below_threshold_count
    return stats_above, stats_below

In [9]:
plt.figure()
plt.hist(histogram_bitumen[histogram_bitumen>0].ravel(), bins=100)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
cmap = plt.cm.get_cmap('jet',20)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111,projection="3d")

xx, yy, zz = np.where(histogram_bitumen >= 11.265)
pnt3d = ax.scatter(xx, yy, zz, c=histogram_bitumen[histogram_bitumen>=11.265], 
                   cmap=cmap, s=10, alpha=.9)
cbar=plt.colorbar(pnt3d, shrink=0.6)
ax.set_xlabel('Z')
ax.set_ylabel('X')
ax.set_zlabel('Y')
ax.view_init(elev=10, azim=20)
plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
cmap = plt.cm.get_cmap('jet',20)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111,projection="3d")

xx, yy, zz = np.where(histogram_bubble >= 11.265)
pnt3d = ax.scatter(xx, yy, zz, c=histogram_bubble[histogram_bubble>=11.265], 
                   cmap=cmap, s=10, alpha=.9)
cbar=plt.colorbar(pnt3d, shrink=0.6)
ax.set_xlabel('Z')
ax.set_ylabel('X')
ax.set_zlabel('Y')
ax.view_init(elev=10, azim=20)
plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
plt.figure()
plt.hist(histogram_bubble[histogram_bubble>0].ravel(), bins=100, alpha=.5, label='bubble')
plt.hist(histogram_bitumen[histogram_bitumen>0].ravel(), bins=100, alpha=.5, label='bitumen')
plt.axvline(11.265, linestyle='--', linewidth=1.5)

plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …